In [30]:
import pandas as pd
import newspaper
from googlesearch import search
import pdb
import tldextract
from datetime import timedelta


In [133]:
import pandas as pd
import newspaper
from googlesearch import search
import pdb
import argparse
import tldextract
from datetime import timedelta




def get_url_bias(url, corpus):
    
    try:
        return corpus.loc[corpus['source_url_processed'].str.contains(tldextract.extract(url).domain)]['bias'].values[0]
    except IndexError:
        return 'center'
    
def get_url_fact(url, corpus):
    try:
        return corpus.loc[corpus['source_url_processed'].str.contains(tldextract.extract(url).domain)]['fact'].values[0]
    except IndexError:
        return 'MIXED'
    
def get_search_query(url):

    article = newspaper.Article(url)

    article.download()
    article.parse()
    article.nlp()

    query             = article.title
    date_before       = article.publish_date + timedelta(days=2)
    date_after        = article.publish_date - timedelta(days=2)

    query_time_before = str(date_before.year) +\
                        '-' + str(date_before.month) +\
                        '-' + str(date_before.day)
            
    query_time_after = str(date_after.year) +\
                        '-' + str(date_after.month) +\
                        '-' + str(date_after.day)
            
            
    query = query + ' before:' + query_time_before + ' after:' + query_time_after
        
    return query
    

    
def get_query_results(search_query):
    
    alt_article_lists = [i for i in search(search_query, num = 10)]
    search_results_df = pd.DataFrame(columns = ['link', 'domain', 'title', 'content'])

    search_results_df['link'] = alt_article_lists
    search_results_df['domain'] = search_results_df['link'].apply(lambda x: tldextract.extract(x).domain\
                                                                             + '.' + tldextract.extract(x).suffix)
    
    return search_results_df


def parse_params():

    parser = argparse.ArgumentParser(description='Source Reliability')
    parser.add_argument('--url',             type=str, default='')
    params = parser.parse_args()
    return params




def main():

    
    user_params = parse_params()
    url         = user_params.url
    corpus      = pd.read_csv('data/corpus.csv')

    url_bias      = get_url_bias(url, corpus)
    url_fact      = get_url_fact(url, corpus)
    
    query         = get_search_query(url)
    query_results_df = get_query_results(query)
    query_results_df = query_results_df.fillna('')


    search_results_df = pd.merge(corpus, query_results_df, left_on = 'source_url_processed', right_on = 'domain')
    search_results_df = search_results_df[~search_results_df['bias'].str.replace('-', ' ').str.contains('right')]
    
    search_results_df.to_csv('results/search_results.csv')

    return search_results_df.to_dict(orient = 'index')


def get_alternative_links(url):

    
    corpus      = pd.read_csv('data/corpus.csv')

    url_bias      = get_url_bias(url, corpus)
    url_fact      = get_url_fact(url, corpus)
    
    query         = get_search_query(url)
    
    print ('The query is:' + (query))
    
    query_results_df = get_query_results(query)
    query_results_df = query_results_df.fillna('')


    search_results_df = pd.merge(corpus, query_results_df, left_on = 'source_url_processed', right_on = 'domain')
    search_results_df = search_results_df[~search_results_df['bias'].str.replace('-', ' ').str.contains('right')]
    

    return search_results_df.to_dict(orient = 'index')



In [134]:
get_alternative_links('http://fox13now.com/2013/12/30/new-year-new-laws-obamacare-pot-guns-and-drones/')

The query is:New Year, new laws: Obamacare, pot, guns and drones before:2014-1-1 after:2013-12-28


{1: {'URL': 'https://mediabiasfactcheck.com/the-peoples-cube/',
  'bias': 'center',
  'content': '',
  'domain': 'thepeoplescube.com',
  'fact': 'LOW',
  'link': 'https://thepeoplescube.com/peoples-blog/america-duck-yeah-victory-merchandise-t12741.html',
  'source_url': 'http://thepeoplescube.com',
  'source_url_processed': 'thepeoplescube.com',
  'title': ''},
 4: {'URL': 'http://mediabiasfactcheck.com/cnn/',
  'bias': 'left',
  'content': '',
  'domain': 'cnn.com',
  'fact': 'MIXED',
  'link': 'http://www.cnn.com/sitemaps/sitemap-articles-2013-12.xml',
  'source_url': 'http://www.cnn.com/',
  'source_url_processed': 'cnn.com',
  'title': ''},
 5: {'URL': 'https://mediabiasfactcheck.com/ndtv/',
  'bias': 'left-center',
  'content': '',
  'domain': 'ndtv.com',
  'fact': 'HIGH',
  'link': 'http://archives.ndtv.com/articles/2014-01.html',
  'source_url': 'https://www.ndtv.com/',
  'source_url_processed': 'ndtv.com',
  'title': ''},
 6: {'URL': 'http://mediabiasfactcheck.com/world-news/',

NameError: name 'dkj' is not defined